In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('features.csv')

In [3]:
import torch

In [ ]:
class CAE(torch.nn.Module):
    def __init__(self):
        super(CAE, self).__init()
        self.encoder = torch.nn.Sequential( # input 37x37x1
            
            # conv1
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2), # (37-3)/2+1 = 18
            torch.nn.BatchNorm2d(num_features=16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2), # 18/2=9 => output is 9x9x16
            
            # conv2
            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2), # (9-3)/2+1 = 4            
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2), # 4/2 = 2 => output is 2x2x32 = 128
            
        )
        
        # dense layers
        self.dense1 = torch.nn.Linear(128, 64)
        self.dense2 = torch.nn.Linear(64, 128)
        
        self.decoder = torch.nn.Sequential(
            
            # convt1
            torch.nn.Upsample(scale_factor=2, mode='bilinear')
            torch.nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            
            # convt2
            torch.nn.Upsample(scale_factor=2, mode='bilinear')
            torch.nn.convTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2),
            torch.nn.BatchNorm2d(1),
            torch.nn.ReLU()
        
        )
        
        def forward(self, x):
            x = self.encoder(x)
            x = torch.nn.functional.relu(self.dense1(x.view(x.size(0), -1))) # flatten tensor
            x = torch.nn.functional.relu(self.dense2(x))
            x = self.decoder(x.view(x.size(0), 32, 2, 2)) # reshape tensor to 32x2x2